In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import datetime

In [ ]:
url = 'https://www.nchu.edu.tw/calendar/'
headers = requests.utils.default_headers()
# https://stackoverflow.com/questions/67809815/remote-end-closed-connection-without-response
# 填寫HTTP header,證明自己為使用者
headers.update(
    {'User-Agent': 'My User Agent 1.0',
     }
)
#https://stackoverflow.com/questions/10667960/python-requests-throwing-sslerror
#ssl取消認證(可能有安全性問題)
html = requests.get(url, verify = False,headers=headers)
html.encoding = 'UTF-8'
sp = BeautifulSoup(html.text, 'lxml')
print(sp.title)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nchu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<title>行事曆 - NCHU,國立中興大學 - 國立中興大學(National Chung Hsing University)</title>


In [ ]:
title1 = sp.find('a',id='top1')
title2 = sp.find('a',id='top2')
#title1=str(title1).replace('<span lang="EN-US">','')#python內建replace用法
#title1=title1.replace("</span>","")
title1=re.sub(r'<[a-zA-Z ="0-9/-]*>',"",str(title1))#用正規表示式更方便
print(title1)
title2=re.sub(r'<[a-zA-Z ="0-9/-]*>',"",str(title2))#用正規表示式更方便
print(title2)

國立中興大學111學年度第1學期行事曆
國立中興大學111學年度第2學期行事曆


In [ ]:
def toCE(string,add=0):
  year = re.findall(r'[0-9]{3}',string)[0]
  year = int(year)+add+1911
  return int(year)

In [ ]:
list1 = sp.find_all('span', style="letter-spacing: -.3pt")#找到行事曆中span標籤的所在地
#將不必要的字符丟棄(<br/>、 </span>)
for i in range(len(list1)):
  list1[i] = str(list1[i]).split("<br/>\r\n")
print(list1)
for i in range(len(list1)):
  for j in range(len(list1[i])):
    list1[i][j] = list1[i][j].replace(" </span>","")
#轉換成numpy矩陣
arr= np.array(list1,dtype=object)
#將矩陣攤平成一維矩陣
arr=arr.flatten()
final=[]
vocation = np.array([])
for i in range(len(arr)):
  for j in range(len(arr[i])):
    if(re.findall(r'.[^寒暑]假.',arr[i][j]) and re.findall(r':[0-9].*[0-9]',arr[i][j])):#用正規表示是找到有放假的"假"字的字串
      date = re.findall(r':[0-9].*[0-9]',arr[i][j])[0]
      length = len(date)
      date = date[-length+1:]#將字串修剪成只有日期
      date=date.replace("、"," ")
      date=date.replace("-"," ")
      date=date.split(" ")
      for k in range(len(date)):
        date[k]=date[k].split('/')
        for f in range(len(date[k])):
          date[k][f]=int(date[k][f])
      final.append(date)
print(final)

[['<span lang="EN-US" style="letter-spacing: -.3pt"> (1)學期開始:8/1', '(2)學士班新生報到(第一梯次)、學士班及研究生新生辦理各類學雜費減免:7/25-8/2', '(3)進修學士班2-5年級網路初選:8/15-8/16', '(4)學士班新生報到(第二梯次:限指考生) 、辦理各類學雜費減免(限指考生):8/17-8/24', '(5)學士班及進修學士班通識課程預選:8/19-8/22', '(6)學士班通識課程初選抽籤:8/25', '(7)開放學雜費繳費(除學士班新生):8/26', '(8)全部學制學生(大一新生除外)就學貸款網站開放登錄及收件期間:8/26-9/19', '(9)進修學士班新生(含轉學生)網路初選:8/29', '(10)學士班各年級(含新生、轉學生)及研究生網路初選:8/29-9/2', '(11)新生進住宿舍:8/30-9/1 </span>'], ['<span lang="EN-US" style="letter-spacing: -.3pt"> (1)新生入學指導:9/2-9/3', '(2)全校學生開學、開始上課(註冊日)、開放學士班新生繳費:9/5', '(3)學士班、進修學士班及研究生(含碩專、產專生)網路加退選:9/5-9/9', '(4)學士班新生就學貸款網站開放及收件期間、全校抵免學分、輔系雙主修、學分學程及第二專長申請:9/5-9/19', '(5)9/9中秋節補假、9/10中秋節(放假):9/9-9/10', '(6)國家防災日地震避難預演:9/14', '(7)全校學生繳費截止:9/19', '(8)國家防災日地震避難演練:9/21', '(9)研究生論文口試截止、研究生畢業離校截止日:9/30 </span>'], ['<span lang="EN-US" style="letter-spacing: -.3pt"> (1)學士班應屆畢業生逕修讀博士學位申請:10/3-10/21', '(2)弱勢學生助學計畫助學金網路開放申請:10/4-10/20', '(3)10/10國慶日(放假):10/10', '(4)加退選學分費開放繳費、課程退選截止:10/14', '(5)上課達1/3 (第6週止):10/15', '(6)校園路跑、啦啦隊比賽、加退選

In [ ]:

def count_vocation(voc):
  if voc[0][0] >=2 and voc[0][0] <8:
    year = toCE(title1,1)
  else:
    year = toCE(title1)
  l = len(voc)
  if l == 1:
    print("哭哭只有一天可以休息{}/{}".format(voc[0][0],voc[0][1]))
  elif (l==2):
    v1 = datetime.date(year,voc[0][0],voc[0][1])
    v2 = datetime.date(year,voc[1][0],voc[1][1])
    if(voc[0][0]==voc[1][0]):
      print("放{}天假".format(abs(v1-v2).days+1))
      print(v1,v2)
  elif l==3:
    v1 = datetime.date(year,voc[0][0],voc[0][1])
    v2 = datetime.date(year,voc[1][0],voc[1][1])
    v3 = datetime.date(year,voc[2][0],voc[2][1])
    print("放{}天假".format(abs(v1-v2).days+1))
    print(v1,v2,v3)
  elif l==4:
    v1 = datetime.date(year,voc[0][0],voc[0][1])
    v2 = datetime.date(year,voc[1][0],voc[1][1])
    print("放{}天假".format(abs(v1-v2).days+1))
    print(v1,v2)
    v3 = datetime.date(year,voc[2][0],voc[2][1])
    v4 = datetime.date(year,voc[3][0],voc[3][1])
    print("放{}天假".format(abs(v3-v4).days+1))
    print(v3,v3)


In [ ]:
for i in range(len(final)):
  count_vocation(final[i])
  print(final[i])

放2天假
2022-09-09 2022-09-10
[[9, 9], [9, 10]]
哭哭只有一天可以休息10/10
[[10, 10]]
放2天假
2022-01-01 2022-01-02
[[1, 1], [1, 2]]
放7天假
2022-01-03 2022-01-09
[[1, 3], [1, 9]]
放8天假
2022-01-20 2022-01-27
[[1, 20], [1, 27]]
哭哭只有一天可以休息2/4
[[2, 4]]
哭哭只有一天可以休息2/18
[[2, 18]]
放2天假
2023-02-27 2023-02-28
[[2, 27], [2, 28]]
哭哭只有一天可以休息3/25
[[3, 25]]
放3天假
2023-04-03 2023-04-05
[[4, 3], [4, 5]]
放2天假
2023-04-06 2023-04-07 2023-04-10
[[4, 6], [4, 7], [4, 10]]
哭哭只有一天可以休息6/17
[[6, 17]]
放2天假
2023-06-22 2023-06-23
[[6, 22], [6, 23]]


In [ ]:
count_vocation([[3,1],[4,4]])